In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002E164E21540>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002E164EA7340>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is enes and I am a Gen AI Engineer")])

AIMessage(content="Hello Enes, it's nice to meet you!\n\nThat's a fascinating field to be in. What kind of work are you doing as a Gen AI Engineer? Are you working on any particular projects you're excited about?\n\nI'm always eager to learn more about the advancements being made in generative AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 23, 'total_tokens': 93, 'completion_time': 0.127272727, 'prompt_time': 0.002208846, 'queue_time': 0.16059641800000002, 'total_time': 0.129481573}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--37ef2288-1be0-4ce0-b1e5-2f43133b031c-0', usage_metadata={'input_tokens': 23, 'output_tokens': 70, 'total_tokens': 93})

In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, my name is enes and I am a Gen AI Engineer"),
        AIMessage("Hello Enes, it's nice to meet you!\n\nThat's a fascinating field to be in. What kind of work are you doing as a Gen AI Engineer? Are you working on any particular projects you're excited about?\n\nI'm always eager to learn more about the advancements being made in generative AI."),
        HumanMessage(content="Hey, what is my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Enes and that you are a Gen AI Engineer. 😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat! 🤖 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 112, 'total_tokens': 162, 'completion_time': 0.090909091, 'prompt_time': 0.005735669, 'queue_time': 0.166710444, 'total_time': 0.09664476}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c6c78b9-6b60-4a00-b9f4-ef5cd7a680a2-0', usage_metadata={'input_tokens': 112, 'output_tokens': 50, 'total_tokens': 162})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [12]:
!pip install langchain_community

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [15]:
config = {"configurable":{"session_id":"chat1"}}

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is enes and I am a Gen AI Engineer")],
    config=config
)

In [18]:
response.content

"Hi Enes, it's nice to meet you!\n\nThat's a fascinating field to be in.  What kind of work are you doing as a Gen AI Engineer?  I'm always eager to learn more about how people are using large language models like me.\n"

In [19]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='You told me your name is Enes! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 97, 'total_tokens': 122, 'completion_time': 0.045454545, 'prompt_time': 0.005299208, 'queue_time': 0.161741841, 'total_time': 0.050753753}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--414cc2e6-277d-4660-b547-6f6aacc6bcab-0', usage_metadata={'input_tokens': 97, 'output_tokens': 25, 'total_tokens': 122})

In [20]:
# change the config
config1 = {"configurable":{"session_id":"chat2"}}

with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1,
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 15, 'total_tokens': 58, 'completion_time': 0.078181818, 'prompt_time': 0.001953886, 'queue_time': 0.160294296, 'total_time': 0.080135704}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--8d3dbb4d-3dfc-4736-9e9c-33e3620bd10c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 43, 'total_tokens': 58})

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Henry Turner")],
    config=config1
)
response.content

'Nice to meet you, Henry Turner!  \n\nWhat can I do for you today?  😊\n'

In [22]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1,
)

AIMessage(content="Your name is Henry Turner,  remember?  😊  \n\nIs there anything else you'd like to talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 109, 'total_tokens': 137, 'completion_time': 0.050909091, 'prompt_time': 0.005116549, 'queue_time': 0.161660384, 'total_time': 0.05602564}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--55b19571-dfff-400a-8bef-136042400fdd-0', usage_metadata={'input_tokens': 109, 'output_tokens': 28, 'total_tokens': 137})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [28]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Enes")]})

AIMessage(content="Hello Enes! \n\nIt's nice to meet you.  Ask me anything - I'll do my best to help! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 32, 'total_tokens': 66, 'completion_time': 0.061818182, 'prompt_time': 0.002344095, 'queue_time': 0.161290765, 'total_time': 0.064162277}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0a7dc215-964e-4533-a669-d5c2037a5173-0', usage_metadata={'input_tokens': 32, 'output_tokens': 34, 'total_tokens': 66})

In [29]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [31]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Enes")],
    config=config,
)

response

AIMessage(content="Hi Enes, \n\nIt's lovely to meet you!  \n\nIs there anything I can do for you today?  I'm ready for your questions or tasks. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 154, 'total_tokens': 196, 'completion_time': 0.076363636, 'prompt_time': 0.007414895, 'queue_time': 0.161275129, 'total_time': 0.083778531}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--25dc31f9-d6ab-4be0-a485-9fe7b6e28e98-0', usage_metadata={'input_tokens': 154, 'output_tokens': 42, 'total_tokens': 196})

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Enes!  😊  \n\nI remember from our earlier conversation. \n\n\nIs there anything else I can help you with?\n'

In [33]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke({
    "messages": [HumanMessage(content="Hi My name is Enes")],
    "language": "Turkish"
})
response.content

'Merhaba Enes,\n\nBenim adım Gemma, senin yardımcınım. Nasıl yardımcı olabilirim? 😊 \n'

In [43]:
with_message_history=RunnableWithMessageHistory( chain,  get_session_history,input_messages_key="messages")

In [46]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I'm enes ")], "language": "Turkish"},
    config=config
)
response.content

'Merhaba Enes!  \n\nNasıl yardımcı olabilirim?  😄 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [60]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [61]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
         | prompt
         | model
         )
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"Turkish"
    }
)
response.content

'Sen vanilya dondurmasını seviyorsun!\n\n\n\n'

In [62]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2" 😊  \n\n\n\n'

In [63]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [64]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob! 😊  \n\n\n\n'

In [65]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked: "whats 2 + 2"  😄'